## Sample with nutpie through onnx

In [1]:
# !!!!!!
# Update this path...
# Should be created by `pixi run install-nutpie`
%env ORT_DYLIB_PATH=/home/adr/.cache/ort.pyke.io/dfbin/x86_64-unknown-linux-gnu/C23A69F709DF91D2BB185D76A29901BD4BE81CD66D85DBCFC5E8BCD851DE9891/onnxruntime/lib/libonnxruntime.so

import nutpie
import torch
import io

env: ORT_DYLIB_PATH=/home/adr/.cache/ort.pyke.io/dfbin/x86_64-unknown-linux-gnu/C23A69F709DF91D2BB185D76A29901BD4BE81CD66D85DBCFC5E8BCD851DE9891/onnxruntime/lib/libonnxruntime.so


In [2]:
N = 10_000

class Module(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mu = torch.nn.Parameter(torch.ones(N), requires_grad=False)

    def forward(self, x):
        return ((-((x - self.mu) ** 2).sum() / 2), (self.mu - x))
        
mod = Module()

In [3]:
compiled = nutpie.compile_pytensor_module(mod, N)
compiled = compiled.with_providers(["cuda"])

/home/adr/git/experiment-nutpie-gpu/.pixi/envs/default/lib/python3.11/site-packages/torch/onnx/_internal/exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [4]:
%%time
tr = nutpie.sample(compiled, chains=6, cores=6)

Progress,Draws,Divergences,Step Size,Gradients/Draw
,1300,0,0.18,15
,1300,0,0.18,31
,1300,0,0.17,31
,1300,0,0.18,31
,1300,0,0.17,15
,1300,0,0.18,15


CPU times: user 1min 10s, sys: 3.77 s, total: 1min 14s
Wall time: 14.7 s


In [5]:
tr.sample_stats.n_steps.sum("draw")

<xarray.DataArray 'n_steps' (chain: 6)> Size: 48B
array([15000, 31000, 31000, 31000, 15000, 15000], dtype=uint64)
Coordinates:
  * chain    (chain) int64 48B 0 1 2 3 4 5

In [6]:
tr.warmup_sample_stats.n_steps.sum("draw")

<xarray.DataArray 'n_steps' (chain: 6)> Size: 48B
array([8659, 8469, 9303, 9709, 9125, 8493], dtype=uint64)
Coordinates:
  * chain    (chain) int64 48B 0 1 2 3 4 5

## Sample with numpyro through pymc

In [7]:
import jax
import pymc as pm

In [8]:
N = 10_000

In [9]:
with pm.Model() as model:
    pm.Normal("x", shape=N)

In [10]:
%%time
with model:
    tr_numpyro = pm.sample(
        nuts_sampler="numpyro",
        discard_tuned_samples=False,
        chains=6,
    )

/home/adr/git/experiment-nutpie-gpu/.pixi/envs/default/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/adr/git/experiment-nutpie-gpu/.pixi/envs/default/lib/python3.11/site-packages/pymc/sampling/jax.py:470: UserWarning: There are not enough devices to run parallel chains: expected 6 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(6)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  pmap_numpyro = MCMC(
sample: 100%|██████████| 2000/2000 [00:27<00:00, 72.13it/s, 31 steps of size 1.61e-01. acc. prob=0.83]


CPU times: user 3min 26s, sys: 7.62 s, total: 3min 33s
Wall time: 3min 27s


In [11]:
# Does not include tuning...
tr_numpyro.sample_stats.n_steps.sum("draw")

<xarray.DataArray 'n_steps' (chain: 6)> Size: 48B
array([31000, 31000, 31000, 31000, 31000, 31000])
Coordinates:
  * chain    (chain) int64 48B 0 1 2 3 4 5

In [12]:
%%time
with model:
    tr_blackjax = pm.sample(
        nuts_sampler="blackjax",
        discard_tuned_samples=False,
        # Out of memory for me if I sample 6 chains...
        nuts_sampler_kwargs=dict(chain_method='vectorized'),
        chains=2,
    )

Running window adaptation


We recommend running at least 4 chains for robust computation of convergence diagnostics
INFO:pymc.stats.convergence:We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
INFO:pymc.stats.convergence:The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


CPU times: user 57.6 s, sys: 1.26 s, total: 58.8 s
Wall time: 57.5 s
